In [ ]:
# Generate data to infer the rules for integral

import json
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot, evalf, N
from IPython.display import display
from utils.utils import *


def remove_constants(f):
    t = Symbol('t')
    return f.as_independent(t)[1]

def round_constants_to_near_integer(expr):
    constants = get_all_constants(expr)
    replacements = {}
    for constant in constants:
        s = str(constant)
        if (".99" in s) or (".98" in s) or (".01" in s) or (".02" in s):
            rounded = constant.round(0)
            replacements[constant] = rounded
    return expr.subs(replacements)

@timeout(5)
# result is a json dict parsed from the line
def parse_row(result):
    try:
        original = round_all_floats(N(sympify(result["original"])))
        integral = round_constants_to_near_integer(
            #remove_constants(
            round_all_floats(N(sympify(result["rounded_regressed"])))
            #)
        )
        #original = filter_non_polynomial(original)
        #integral = filter_non_polynomial(integral)
        original_integral = integrate(original, t)
        avg_diff = get_avg_diff(original_integral, integral, t)
        if avg_diff > MAX_AVG_DIFF or len(original_integral.args) != len(integral.args):
            #print("Skipping. Diff=", avg_diff)
            #display(original)
            #display(original_integral)
            #display(integral)
            return None, None
    except:
        print("Cannot process", str(integral))
        return None, None
    return original, integral

fin = open("datasets/parametric_equations_randomized_nonpoly_integral_results.json", "r")
lines = fin.readlines()
fin.close()

MAX_POWER = 6
MAX_AVG_DIFF = 0.01

originals = []
integrals = []
t = Symbol('t')

for i in range(len(lines)):
    if i % 100 == 0:
        print(i, "rows processed")
    line = lines[i]
    result = json.loads(line)
    if "rounded_regressed" not in result:
        continue
    #print(line)
    try:
        original, integral = parse_row(result)
    except:
        print("Cannot process", str(integral))
        continue
    if integral is None: continue
    originals.append(str(original))
    integrals.append(str(integral))
    if len(originals) % 100 == 0:
        print(len(originals), "valid cases loaded")
    
fin.close()

sentences = []
for i in range(len(originals)):
    sentences.append(originals[i] + ' entail ' + integrals[i] + ' end')
fin.close()


0 rows processed
100 rows processed
100 valid cases loaded
200 rows processed
300 rows processed
200 valid cases loaded
400 rows processed
Cannot process -1.0*exp(-1.0*t)
Cannot process None
Cannot process None
500 rows processed
Cannot process -1.0*cos(1.0*t)
Cannot process 3.4*t
Cannot process -5.0*t + 0.29*cos(3.4*t)
300 valid cases loaded
600 rows processed
700 rows processed
400 valid cases loaded
800 rows processed
900 rows processed
500 valid cases loaded
1000 rows processed
1100 rows processed
600 valid cases loaded
1200 rows processed
700 valid cases loaded
1300 rows processed
1400 rows processed
800 valid cases loaded
1500 rows processed


In [2]:
len(sentences)

63409

In [3]:
expr = sp.sympify("-3.2*t - 0.98*cos(1.0*t)")
round_constants_to_integer(expr)

NameError: name 'round_constants_to_integer' is not defined

In [1]:
import json
import torch
import random
from datasets import load_dataset, Dataset, load_from_disk

# random.shuffle(sentences)

# ds = Dataset.from_dict({'eq_pair': sentences})
# train_ds = ds.train_test_split(test_size=0.025)

# train_ds['train'][1]

# train_ds.save_to_disk("datasets/integrate_gptneo_nonpoly_dataset_63k")

train_ds = load_from_disk("datasets/integrate_gptneo_nonpoly_dataset_63k")

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_ds

DatasetDict({
    train: Dataset({
        features: ['eq_pair'],
        num_rows: 61823
    })
    test: Dataset({
        features: ['eq_pair'],
        num_rows: 1586
    })
})

In [3]:
from transformers import AutoTokenizer

CONTEXT_LENGTH = 256

tokenizer = AutoTokenizer.from_pretrained("xhyi/PT_GPTNEO350_ATG") #"EleutherAI/gpt-neo-125m") 
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    return tokenizer(examples["eq_pair"], padding='max_length', truncation=True, max_length=CONTEXT_LENGTH, return_tensors="pt")

tokenized_ds = train_ds.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=train_ds["train"].column_names,
)

def preprocess_function(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

lm_dataset = tokenized_ds.map(preprocess_function, batched=True, num_proc=1)

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1586/1586 [00:00<00:00, 15275.69 examples/s]


In [ ]:
# from transformers import AutoTokenizer

# CONTEXT_LENGTH = 256

# tokenizer = AutoTokenizer.from_pretrained("xhyi/PT_GPTNEO350_ATG")

# tokens = tokenizer.encode("0.1428571428571")
# for token in tokens:
#     print(tokenizer.decode([token]))


In [ ]:
# for i in range(len(lm_dataset['train'])):
#     if len(lm_dataset['train'][i]['input_ids']) != 256:
#         print(i, len(lm_dataset['train'][i]['input_ids']))
#     if len(lm_dataset['train'][i]['labels']) != 256:
#         print(i, len(lm_dataset['train'][i]['labels']))

In [6]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("datasets/amps_mathematica_algebra_numtheory_gptneo_350m/gptneo-350m-985000.model")  #"EleutherAI/gpt-neo-125m")
# config = model0.config

# model = AutoModelForCausalLM.from_config(config)
# model0 = None

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="datasets/integrate_gptneo_350m_w_pretrain_nonpoly_63k_202401",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    fp16=False,
    save_steps=2000,
    eval_steps=2000,
    logging_steps=2000,
    save_total_limit=20,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
2000,0.542400,0.439209
4000,0.414800,0.404192
6000,0.386500,0.391458
8000,0.378200,0.382247
10000,0.362900,0.377428
12000,0.358500,0.376084
14000,0.346100,0.373145
16000,0.340000,0.372355
18000,0.328100,0.371064


Checkpoint destination directory datasets/integrate_gptneo_350m_w_pretrain_nonpoly_63k_202401/checkpoint-18000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=19320, training_loss=0.38026400106047004, metrics={'train_runtime': 13796.3603, 'train_samples_per_second': 22.406, 'train_steps_per_second': 1.4, 'total_flos': 1.4350265149292544e+17, 'train_loss': 0.38026400106047004, 'epoch': 5.0})

In [10]:
model.save_pretrained("datasets/integrate_gptneo_350m_w_pretrain_nonpoly_63k_202401/gptneo-350m-19000.model")

In [ ]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"

# model = AutoModelForCausalLM.from_pretrained("datasets/integrate_gptneo_202401/gptneo-350m-5500-loss0.350.model").to(device)

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

def generate_integral(input):
    # Encode some input text
    prompt = input + " entail"
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    # Generate text
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7)
    # Decode and print the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    #print(generated_text)
    if 'entail' in generated_text:
        generated_text = generated_text[generated_text.find('entail') + 6:].strip()
        #print(generated_text)
    if 'end' in generated_text:
        generated_text = generated_text[0:generated_text.find('end')].strip()
    return generated_text
    
print(generate_integral("2.5*sin(4*t)"))

NameError: name 'model' is not defined

In [16]:
# import sympy as sp
# from utils.utils import *

# MIN_ALLOWED_DIFF = 0.011

# test_ds = train_ds['test']

# num_processed = 0
# num_equal = 0
# num_zero_diff = 0
# num_within_allowed = 0
# t = sp.Symbol('t')

# verbose = False

for i in range(min(len(test_ds), 100)):
    if verbose:
        print("Case", i, test_ds[i]['eq_pair'])
    eq_pair = test_ds[i]['eq_pair']
    question = eq_pair[0:eq_pair.find('entail')].strip()
    answer = eq_pair[eq_pair.find('entail') + 6:].replace('end', '').strip()
    if 'repeat' in question:
        question = question[0:question.find('repeat')]
    print(eq_pair)
    original = sp.sympify(question)
    integral = round_all_floats(sp.integrate(original, t), 2)
    if verbose: display(integral)
    try:
        pred = generate_integral(question)
        generated = round_all_floats(sp.sympify(pred), 2)
        if verbose: display(generated)
    except:
        print("Cannot sympify", pred)
        continue
    avg_diff = get_avg_diff(integral, generated, t)
    if verbose: print("avg_diff", avg_diff)
    num_processed += 1
    if avg_diff <= MIN_ALLOWED_DIFF:
        num_within_allowed += 1
    else:
        display(integral)
        display(generated)
        print(avg_diff)
    if avg_diff <= 0.0000001:
        num_zero_diff += 1
    diff_expr = sp.simplify(integral-generated)
    if verbose: display(diff_expr)
    if diff_expr == 0:
        if verbose: print("Equal")
        num_equal += 1
    if i%100 == 0:
        print(i+1, "processed")
        
# print("num_processed", num_processed)
# print("num_equal", num_equal)
# print("num_zero_diff", num_zero_diff)
# print("num_within_allowed", num_within_allowed)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2.6*exp(t) entail 2.6*exp(1.0*t) end


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1 processed
6.00000000000000 entail 6.0*t end


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

-0.2*t + 3.83*exp(-1.2*t)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.08593281838120537
3.0*cos(2*t) entail 1.5*sin(2.0*t) end


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3.0*sin(3.4*t) + 2.0 entail 2.0*t - 0.88*cos(3.4*t) end
num_processed 100
num_equal 38
num_zero_diff 90
num_within_allowed 96


In [9]:
from transformers import TrainingArguments, Trainer
import sympy as sp
from utils.utils import *
import time

MIN_ALLOWED_DIFF = 0.011

test_ds = train_ds['test']

training_args = TrainingArguments(
    output_dir="datasets/integrate_gptneo_350m_w_pretrain_nonpoly_63k_202401",
    evaluation_strategy="steps",
    learning_rate=2e-7,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=0.001,
    fp16=True,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=1000,
    save_total_limit=20,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

for step in range(19000, 21000, 2000):
    print("Recovering checkpoint", step)
    cp_path = "datasets/integrate_gptneo_350m_w_pretrain_nonpoly_63k_202401/checkpoint-" + str(step)
    trainer.train(cp_path)
    #
    num_processed = 0
    num_equal = 0
    num_zero_diff = 0
    num_within_allowed = 0
    total_time = 0.0
    t = sp.Symbol('t')
    #
    verbose = False
    #
    for i in range(min(len(test_ds), 1000)):
        if verbose:
            print("Case", i, test_ds[i]['eq_pair'])
        eq_pair = test_ds[i]['eq_pair']
        question = eq_pair[0:eq_pair.find('entail')].strip()
        answer = eq_pair[eq_pair.find('entail') + 6:].replace('end', '').strip()
        if 'repeat' in question:
            question = question[0:question.find('repeat')]
        if verbose: print(eq_pair)
        original = sp.sympify(question)
        integral = round_all_floats(sp.integrate(original, t), 2)
        if verbose: display(integral)
        try:
            t1 = time.time()
            pred = generate_integral(question)
            generated = round_all_floats(sp.sympify(pred), 2)
            t2 = time.time()
            total_time += (t2-t1)
            if verbose: display(generated)
        except:
            print("Cannot sympify", pred)
            continue
        avg_diff = get_avg_diff(integral, generated, t)
        if verbose: print("avg_diff", avg_diff)
        num_processed += 1
        if avg_diff <= MIN_ALLOWED_DIFF:
            num_within_allowed += 1
        else:
            display(integral)
            display(generated)
            print(avg_diff)
        if avg_diff <= 0.0000001:
            num_zero_diff += 1
        diff_expr = sp.sympify("t+1")
        try:
            diff_expr = sp.simplify(integral-generated)
        except:
            pass
        if verbose: display(diff_expr)
        if diff_expr == 0:
            if verbose: print("Equal")
            num_equal += 1
        if i%100 == 0:
            print(i+1, "processed")
    #
    for i in range(10): print()
    print("num_processed", num_processed)
    print("num_equal", num_equal)
    print("num_zero_diff", num_zero_diff)
    print("num_within_allowed", num_within_allowed)
    print("avg. time =", total_time/num_processed)
    for i in range(10): print()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You are resuming training from a checkpoint trained with 4.36.1 of Transformers but your current version is 4.36.2. This is not recommended and could yield to errors or unwanted behaviors.
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Recovering checkpoint 19000


Step,Training Loss,Validation Loss


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1 processed


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.16666666666666669


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul











num_processed 1000
num_equal 374
num_zero_diff 872
num_within_allowed 943
avg. time = 0.7491092052459717










